In [1]:
import requests
import pandas as pd
import json

In [2]:
path ='opendns-random-domains.txt'
df = pd.read_csv(path, skiprows=0, header=None, sep='\n', delim_whitespace=False)

In [3]:
df.head()

,0
0,webmagnat.ro
1,nickelfreesolutions.com
2,scheepvaarttelefoongids.nl
3,tursan.net
4,plannersanonymous.com


In [4]:
index_list = ["2018-34"]

In [5]:
### -----------------------
### Searches the Common Crawl Index for a domain.
### using 2018-34
### -----------------------
def search_domain(domain):
    record_list = []
    print("Trying target domain: ", domain)
    
    for index in index_list:
        print("Trying index ", index)
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-"+index+"-index?url="+domain+"&matchType=domain&output=json"        
        #http://index.commoncrawl.org/CC-MAIN-2018-34-index?url=webmagnat.ro&output=json
        print("Trying cc url ", cc_url)
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            records = response.content.splitlines()
            for record in records:
                record_list.append(json.loads(record))  
            print("Added %d results. ", len(records))
    print("Found a total of %d hits. ", len(record_list))
    return record_list

In [ ]:
test = df[:3]
print(test)

In [8]:
not_phish = []
for index, row in df.iterrows():
    record_list = search_domain(row[0])
    not_phish.append(record_list)

Trying target domain:  webmagnat.ro
Trying index  2018-34
Trying cc url  http://index.commoncrawl.org/CC-MAIN-2018-34-index?url=webmagnat.ro&matchType=domain&output=json
Added %d results.  42
Found a total of %d hits.  42
Trying target domain:  nickelfreesolutions.com
Trying index  2018-34
Trying cc url  http://index.commoncrawl.org/CC-MAIN-2018-34-index?url=nickelfreesolutions.com&matchType=domain&output=json
Found a total of %d hits.  0
Trying target domain:  scheepvaarttelefoongids.nl
Trying index  2018-34
Trying cc url  http://index.commoncrawl.org/CC-MAIN-2018-34-index?url=scheepvaarttelefoongids.nl&matchType=domain&output=json
Added %d results.  4
Found a total of %d hits.  4
Trying target domain:  tursan.net
Trying index  2018-34
Trying cc url  http://index.commoncrawl.org/CC-MAIN-2018-34-index?url=tursan.net&matchType=domain&output=json
Found a total of %d hits.  0
Trying target domain:  plannersanonymous.com
Trying index  2018-34
Trying cc url  http://index.commoncrawl.org/CC-

In [9]:
import json
with open('not_phish.json', 'w') as outfile:
    json.dump(not_phish, outfile)

In [10]:
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df

In [11]:
test = df[:3]
print(test)
test_phish = []
for index, row in test.iterrows():
    test_list = search_domain(row[0])
    test_phish.append(test_list)
    
with open('test_phish.json', 'w') as outfile:
    json.dump(test_phish, outfile)

                            0
0                webmagnat.ro
1     nickelfreesolutions.com
2  scheepvaarttelefoongids.nl
Trying target domain:  webmagnat.ro
Trying index  2018-34
Trying cc url  http://index.commoncrawl.org/CC-MAIN-2018-34-index?url=webmagnat.ro&matchType=domain&output=json
Added %d results.  42
Found a total of %d hits.  42
Trying target domain:  nickelfreesolutions.com
Trying index  2018-34
Trying cc url  http://index.commoncrawl.org/CC-MAIN-2018-34-index?url=nickelfreesolutions.com&matchType=domain&output=json
Found a total of %d hits.  0
Trying target domain:  scheepvaarttelefoongids.nl
Trying index  2018-34
Trying cc url  http://index.commoncrawl.org/CC-MAIN-2018-34-index?url=scheepvaarttelefoongids.nl&matchType=domain&output=json
Added %d results.  4
Found a total of %d hits.  4


In [97]:
while '' in test_phish:
    test_phish.remove('')
test_pd = json_normalize(test_phish[0])

In [105]:
test_pd.dtypes

charset          object
digest           object
filename         object
languages        object
length           object
mime             object
mime-detected    object
offset           object
status           object
timestamp        object
url              object
urlkey           object
dtype: object

In [ ]:
i = 1
while i < len(test_phish):
    print("i = ",i)
    norms = pd.DataFrame(json_normalize(test_phish[i]))
    print("len of norm = ",len(norms))
    print(norms.dtypes)
    print(test_pd.dtypes)
    #test_pd += norms
    #for norm in norms:
    test_pd = test_pd.append(norms)
    print("len of test = ",len(test_pd))
    i += 1

In [119]:
len(test_pd)

42

In [50]:
len(not_phish)

10000

In [63]:
while '' in not_phish:
    not_phish.remove('')

In [71]:
not_phish_pd = json_normalize(not_phish[0])

In [70]:
json_normalize(not_phish[4])

,digest,filename,length,mime,mime-detected,offset,status,timestamp,url,urlkey
0,3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ,crawl-data/CC-MAIN-2018-34/segments/1534221209...,632,text/html,text/html,6356482,301,20180815071108,http://plannersanonymous.com/,"com,plannersanonymous)/"
1,XNFFJ6VAXWKDEXG4RE4KTA2W6ENTMXMG,crawl-data/CC-MAIN-2018-34/segments/1534221209...,642,text/html,text/html,911743,404,20180815071105,http://plannersanonymous.com/robots.txt,"com,plannersanonymous)/robots.txt"


In [ ]:
i = 1
while i < len(not_phish):
    print(i)
    norms = pd.DataFrame(json_normalize(not_phish[i]))
    #norm = json_normalize(not_phish[i])
    not_phish_pd = not_phish_pd.append(norms)
    print(len(not_phish_pd))
    i += 1

In [53]:
not_phish_pd.head()

,charset,digest,filename,languages,length,mime,mime-detected,offset,status,timestamp,url,urlkey
0,NaN,3DN75HT35NOW2WGZG3CIOTAA767OYNUC,crawl-data/CC-MAIN-2018-34/segments/1534221211...,NaN,656,text/html,text/html,15380184,301,20180816182647,http://www.webmagnat.ro/,"ro,webmagnat)/"
1,UTF-8,IZ6CKTZ6FJ57X3GVWTNQ5F3XTWQL6ULP,crawl-data/CC-MAIN-2018-34/segments/1534221211...,"ron,eng",15078,text/html,text/html,1000613528,200,20180816182648,https://www.webmagnat.ro/,"ro,webmagnat)/"
2,NaN,3DN75HT35NOW2WGZG3CIOTAA767OYNUC,crawl-data/CC-MAIN-2018-34/segments/1534221214...,NaN,654,text/html,text/html,15327636,301,20180819023333,http://www.webmagnat.ro/,"ro,webmagnat)/"
3,UTF-8,IZ6CKTZ6FJ57X3GVWTNQ5F3XTWQL6ULP,crawl-data/CC-MAIN-2018-34/segments/1534221214...,"ron,eng",15073,text/html,text/html,984536606,200,20180819023333,https://www.webmagnat.ro/,"ro,webmagnat)/"
4,NaN,3DN75HT35NOW2WGZG3CIOTAA767OYNUC,crawl-data/CC-MAIN-2018-34/segments/1534221215...,NaN,657,text/html,text/html,9314042,301,20180819232223,http://webmagnat.ro/,"ro,webmagnat)/"


In [122]:
not_phish_sv = not_phish_pd['url']

In [60]:
len(not_phish)

10000

In [123]:
not_phish_sv.to_csv("sample_neg_phish.csv",index=False)